In [2]:
import json
import pathlib

In [3]:
from datasets import load_dataset, Dataset

In [4]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from typing import Optional


class EnvSettings(BaseSettings, case_sensitive=True):
    model_config = SettingsConfigDict(env_file="./.env", env_file_encoding="utf-8")
    HUGGINGFACE_TOKEN: Optional[str]

In [18]:
env_settings = EnvSettings()

In [ ]:
data = []
for file in [p for p in pathlib.Path("../data").iterdir() if p.is_file()]:
    with open(file) as f:
        data.extend(json.load(f))
data

In [ ]:
for idx1 in range(len(data)):
    for idx2 in range(len(data[idx1]["dialogues"])):
        data[idx1]["dialogues"][idx2].pop("topic", None)
        data[idx1]["dialogues"][idx2].pop("validate", None)
data

In [10]:
counter = {}
for idx1 in range(len(data)):
    if data[idx1]["topic"] not in counter:
        counter[data[idx1]["topic"]] = 0
    counter[data[idx1]["topic"]] += 1
    for idx2 in range(len(data[idx1]["dialogues"])):
        data[idx1]["dialogues"][idx2]["id"] = (
            f"{data[idx1]['topic']}_{counter[data[idx1]['topic']]}_{idx2}"
        )

In [ ]:
data[5]["dialogues"][1]

In [14]:
dataset = Dataset.from_list(data)

In [15]:
dataset

Dataset({
    features: ['graph', 'topic', 'dialogues'],
    num_rows: 402
})

In [ ]:
dataset[0]["dialogues"][1]

In [ ]:
dataset.push_to_hub(
    "DeepPavlov/d2g_generated", private=True, token=env_settings.HUGGINGFACE_TOKEN
)

In [ ]:
dataset = load_dataset("DeepPavlov/d2g_generated", token=env_settings.HUGGINGFACE_TOKEN)
dataset

In [ ]:
dataset["train"][0]